## Generate MARCXML based on the Library of Congress BIBFRAME2MARC mapping

In [ ]:
%pip install rdflib
%pip install lxml
%pip install requests
%pip install saxonche

## Replace the URI with your Instance URI from Sinopia


In [1]:
# replace uri with the Instance uri of the resource you want to convert
uri = "https://api.stage.sinopia.io/resource/01a226e9-ed48-4185-ad77-94c1723a3fec"

from saxonche import PySaxonProcessor
import lxml.etree as ET
from marc_xml.lc_bfxml_work import lc_bfxml_work, remove_last_line
from marc_xml.lc_bfxml_instance import lc_bfxml_instance, remove_rdf_header


## Generate a combined LoC RDF Work and Instance file

In [4]:
def rdf2marcxml(instance_uri):
    # quality check the file
    if instance_uri is None:
        raise ValueError("No data received")
        
    lc_bfxml_work(instance_uri)  
    remove_last_line() 
    lc_bfxml_instance(instance_uri) 
    remove_rdf_header() 
    
    # combine the two files, work first
    with open("bfxml_work.xml", "r") as work_file:
        work = work_file.read()
    with open("lc_bfxml_instance.xml", "r") as instance_file:
        instance = instance_file.read()
    
    # save as a file
    with open("LoC_Work_Instance.xml", "w") as combined_file:
        combined_file.write(work + instance)

    # add the sinopiabf namespace to the combined file
    with open("LoC_Work_Instance.xml", "r") as file:
        filedata = file.read()
    filedata = filedata.replace('<rdf:RDF', '<rdf:RDF xmlns:sinopiabf="http://sinopia.io/vocabulary/bf/"')
    with open("LoC_Work_Instance.xml", "w") as file:
        file.write(filedata)

    # apply "pre-transform-normalize.xsl" for normalization
    dom = ET.parse("LoC_Work_Instance.xml")
    xslt = ET.parse("marc_xml/xsl/pre-transform-normalize.xsl")
    transform = ET.XSLT(xslt)
    newdom = transform(dom)
    with open("LoC_Work_Instance_Normalized.xml", "w") as f:
        f.write(str(newdom))    

    return newdom

# Call the function with an instance_uri
newdom = rdf2marcxml(uri)
print(newdom)

<?xml version="1.0"?>
<rdf:RDF xmlns:sinopiabf="http://sinopia.io/vocabulary/bf/" xmlns:bf="http://id.loc.gov/ontologies/bibframe/" xmlns:sinopia="http://sinopia.io/vocabulary/" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bflc="http://id.loc.gov/ontologies/bflc/">
  <bf:Work rdf:about="https://api.stage.sinopia.io/resource/187ba9d9-7eb6-47cf-ba02-4e55763e839a">
    <rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Monograph"/>
    <sinopia:hasResourceTemplate>pcc:bf2:Monograph:Work</sinopia:hasResourceTemplate>
    <bf:title>
      <bf:Title rdf:nodeID="ffff4d67462b34194bf6a5a93b0a5b2f2b1">
        <bf:mainTitle xml:lang="en">Hasst du noch alle?! : Gesunde Kekse, leere Versprechen und Partyspiele. - 333 Gründe, um täglich aus der Haut zu fahren</bf:mainTitle>
      </bf:Title>
    </bf:title>
    <bf:contribution>
      <bf:PrimaryContribution rdf:nodeID="ffff4d67462b34194bf6a5a93b0a5b2f2b2">
        <bf:a

## Generate MARCXML from the combined RDF/XML

In [ ]:
# use PySaxonProcessor to appy bibframe2marc.xsl to LoC_Work_Instance_Normalized.xml
with PySaxonProcessor(license=False) as proc:
    xsltproc = proc.new_xslt30_processor()
    document = proc.parse_xml(xml_file_name="LoC_Work_Instance_Normalized.xml")
    executable = xsltproc.compile_stylesheet(stylesheet_file="bibframe2marc.xsl")
    output = executable.transform_to_string(xdm_node=document)
    #save as a file
    with open("marc.xml", "w") as f:
        f.write(output)
    print(output)
